<strong>API</strong> Calls but gives just the location in the memory

In [1]:
import os
import pefile

filename = "BenignTestData/Camera.exe"

print(os.path.basename(filename))
pe = pefile.PE(filename)


for entry in pe.DIRECTORY_ENTRY_IMPORT:
    for API in entry.imports:
        print(API)


Camera.exe


Whole information about the file

In [2]:
pe.print_info()

----------DOS_HEADER----------

[IMAGE_DOS_HEADER]
0x0        0x0   e_magic:                       0x5A4D    
0x2        0x2   e_cblp:                        0x90      
0x4        0x4   e_cp:                          0x3       
0x6        0x6   e_crlc:                        0x0       
0x8        0x8   e_cparhdr:                     0x4       
0xA        0xA   e_minalloc:                    0x0       
0xC        0xC   e_maxalloc:                    0xFFFF    
0xE        0xE   e_ss:                          0x0       
0x10       0x10  e_sp:                          0xB8      
0x12       0x12  e_csum:                        0x0       
0x14       0x14  e_ip:                          0x0       
0x16       0x16  e_cs:                          0x0       
0x18       0x18  e_lfarlc:                      0x40      
0x1A       0x1A  e_ovno:                        0x0       
0x1C       0x1C  e_res:                         
0x24       0x24  e_oemid:                       0x0       
0x26       0x26

https://github.com/trailofbits/pe-parse

API list with name and DLL using the code above.

In [3]:
import pepy

p = pepy.parse(filename)

import_list = p.get_imports()

for item in import_list:
    print(item.sym + " " + item.name )
    print("####################################")


TraceMessage ADVAPI32.DLL
####################################
GetTraceLoggerHandle ADVAPI32.DLL
####################################
GetTraceEnableLevel ADVAPI32.DLL
####################################
GetTraceEnableFlags ADVAPI32.DLL
####################################
RegisterTraceGuidsW ADVAPI32.DLL
####################################
EventRegister ADVAPI32.DLL
####################################
EventSetInformation ADVAPI32.DLL
####################################
EventWrite ADVAPI32.DLL
####################################
OutputDebugStringW KERNEL32.DLL
####################################
SetEvent KERNEL32.DLL
####################################
ReleaseSRWLockExclusive KERNEL32.DLL
####################################
WaitForSingleObjectEx KERNEL32.DLL
####################################
AcquireSRWLockExclusive KERNEL32.DLL
####################################
LeaveCriticalSection KERNEL32.DLL
####################################
InitializeCriticalSectionEx KERNEL32.DLL
#

## REAL IMPLEMENTATION : TRAVERSE ALL THE FILES IN A DIRECTORY AND EXTRACT FEATURES WITH PEFILE

Check bitcoin address in the file with YARA Rules

In [4]:
import yara

def check_bitcoinAdress(file, rulePath):
    
    rule = yara.compile(filepath = rulePath)
    
    m = rule.match(file)
    if m:
        return 1 # found
    else:
        return 0 # not found

Extract features with the pefile module

In [5]:
def extract_features(file):
    # Creates an empty list for which features can later be appended into.
    features = []

    # Name of file
    features.append(os.path.basename(file))

    # Assigns pe to the input file. fast_load loads all directory information.
    pe = pefile.PE(file, fast_load=True)

    # CPU that the file is intended for.
    features.append(pe.FILE_HEADER.Machine)

    # DebugSize is the size of the debug directory table. Clean files typically have a debug directory
    # and thus, will have a non-zero values.
    features.append(pe.OPTIONAL_HEADER.DATA_DIRECTORY[6].Size)

    # DebugRVA
    features.append(pe.OPTIONAL_HEADER.DATA_DIRECTORY[6].VirtualAddress)

    # MajorImageVersion is the version of the file. This is user defined and for clean programs is often
    # populated. Malware often has a value of 0 for this.
    features.append(pe.OPTIONAL_HEADER.MajorImageVersion)

    # MajorOSVersion is the major operating system required to run exe.
    features.append(pe.OPTIONAL_HEADER.MajorOperatingSystemVersion)

    # ExportRVA.
    features.append(pe.OPTIONAL_HEADER.DATA_DIRECTORY[0].VirtualAddress)

    # ExportSize is the size of the export table. Usually non-zero for clean files.
    features.append(pe.OPTIONAL_HEADER.DATA_DIRECTORY[0].Size)

    # IatRVA is the relative virtual address of import address table. Most clean files have 4096 for this
    # where as malware often has 0 or a very large number.
    features.append(pe.OPTIONAL_HEADER.DATA_DIRECTORY[12].VirtualAddress)

    # Version of linker that produced file.
    features.append(pe.OPTIONAL_HEADER.MajorLinkerVersion)
    features.append(pe.OPTIONAL_HEADER.MinorLinkerVersion)

    # NumberOfSections is the number of sections in file.
    features.append(pe.FILE_HEADER.NumberOfSections)

    # SizeOfStackReserve denotes the amount of virtual memory to reserve for the initial thread's stack.
    features.append(pe.OPTIONAL_HEADER.SizeOfStackReserve)

    # DllCharacteristics is a set of flags indicating under which circumstances a DLL's initialization
    # function will be called.
    features.append(pe.OPTIONAL_HEADER.DllCharacteristics)

    # MinResourcesSize is the size of resources section of PE header. Malware sometimes has 0 resources.
    features.append(pe.OPTIONAL_HEADER.DATA_DIRECTORY[2].Size)

    # Calls the check_bitcoinAdress function to check if the file contains a bitcoin address.
    bitcoin_check = check_bitcoinAdress(file, yaraRule_path)
    features.append(bitcoin_check)

    # Returns the feature list.
    return features
    

Traverse all files and extract features. Export them as CSV files seperately both for benign and ransomware files

In [3]:
benign_directory = "BenignTestData"
ransomware_directory = "RansomwareTestData" # ??? 

yaraRule_path = "YARA/bitcoin.yara"

output_file_benign = "data_benign.csv"
output_file_ransomware= "data_ransomware.csv"

csv_delimeter = ','
csv_columns = [
    "FileName",
    "Machine",
    "DebugSize",
    "DebugRVA",
    "MajorImageVersion",
    "MajorOSVersion",
    "ExportRVA",
    "ExportSize",
    "IatVRA",
    "MajorLinkerVersion",
    "MinorLinkerVersion",
    "NumberOfSections",
    "SizeOfStackReserve",
    "DllCharacteristics",
    "ResourceSize",
    "BitcoinAddresses",
    "Benign",
]

def create_dataset_csv(directory, output_file, fileType):
    '''
    directory : path for the dataset
    output_file : path for output
    fileType : "benign" or "ransomware"
    '''
    
    # Opens file so features can be written too.
    feature_file = open(output_file, 'w')

    # Writes column headers to feature file.
    feature_file.write(csv_delimeter.join(csv_columns) + "\n")
    
    if fileType == "benign":
        for item in os.listdir(directory):
            try:
                filename = os.path.join(directory,item)
                features = extract_features(filename)
                features.append(1)
                feature_file.write(csv_delimeter.join(map(lambda x: str(x), features)) + "\n")
                print(features)
            except:
                print("Error occured, this file is not a PE file", item)
            
    elif fileType == "ransomware":
        for item in os.listdir(directory):
            try:
                filename = os.path.join(directory,item)
                features = extract_features(filename)
                features.append(0)
                feature_file.write(csv_delimeter.join(map(lambda x: str(x), features)) + "\n")
                print(features)
            except:
                print("Error occured, this file is not a PE file", item)

In [14]:
create_dataset_csv(benign_directory, output_file_benign, "benign")
''' 
create_dataset_csv(ransomware_directory, output_file_ransomware, "ransomware")
'''

['calc.exe', 332, 56, 376204, 6, 6, 0, 0, 397312, 11, 0, 5, 262144, 33088, 381224, 0, 1]
['Camera.exe', 332, 56, 500544, 0, 6, 663408, 139, 499712, 11, 0, 7, 1048576, 33216, 377608, 0, 1]
['cmd.exe', 332, 56, 151160, 6, 6, 0, 0, 266240, 11, 0, 5, 1048576, 33088, 33872, 0, 1]
['dfrgui.exe', 332, 28, 4128, 6, 6, 0, 0, 86016, 11, 0, 5, 262144, 33088, 473840, 0, 1]
['explorer.exe', 332, 56, 1121668, 6, 6, 0, 0, 1138688, 11, 0, 6, 262144, 32960, 839096, 0, 1]
['FileManager.exe', 332, 56, 24976, 0, 6, 0, 0, 24576, 11, 0, 7, 1048576, 33216, 39120, 0, 1]
['mspaint.exe', 332, 56, 592320, 6, 6, 0, 0, 618496, 11, 0, 5, 262144, 33088, 5743728, 0, 1]
['msra.exe', 332, 28, 4160, 6, 6, 0, 0, 286720, 11, 0, 5, 262144, 33088, 214576, 0, 1]
['notepad.exe', 332, 56, 90412, 6, 6, 0, 0, 106496, 11, 0, 5, 262144, 33088, 105200, 0, 1]
['PhotosApp.exe', 332, 56, 24976, 0, 6, 0, 0, 24576, 11, 0, 7, 1048576, 33216, 298704, 0, 1]
['reg.exe', 332, 56, 53164, 6, 6, 0, 0, 323584, 11, 0, 5, 262144, 33088, 2024, 0, 1

' \ncreate_dataset_csv(ransomware_directory, output_file_ransomware, "ransomware")\n'

Import libraries and add hex code data to the dataframe. Also add combine ransomware and benign data. Basically finalize the dataframe

In [9]:
import pandas as pd
from sklearn import model_selection
import sklearn.ensemble as ske
import sklearn.metrics
from sklearn.metrics import f1_score, classification_report

import disassembler # python file that fetches hex codes

# Creates pandas dataframe and reads in csv file.
df = pd.read_csv('data_benign.csv', sep=',')

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# If output.csv file for hex codes is not exist, uncomment line below
''' 
disassembler.disassemble_files("BenignTestData")
'''
df2 = pd.read_csv('output.csv', sep=',')
df = pd.concat([df, df2], axis=1)
print(df)


# Combine benign and ransomware data, uncomment lines below if you have a dataset in a folder "RansomwareTestData"
''' 
df2 = pd.read_csv('data_ransomware.csv', sep=',')
result = pd.concat([df, df2], axis=0)
df = result
'''

            FileName  Machine  DebugSize  DebugRVA  MajorImageVersion  \
0           calc.exe      332         56    376204                  6   
1         Camera.exe      332         56    500544                  0   
2            cmd.exe      332         56    151160                  6   
3         dfrgui.exe      332         28      4128                  6   
4       explorer.exe      332         56   1121668                  6   
5    FileManager.exe      332         56     24976                  0   
6        mspaint.exe      332         56    592320                  6   
7           msra.exe      332         28      4160                  6   
8        notepad.exe      332         56     90412                  6   
9      PhotosApp.exe      332         56     24976                  0   
10           reg.exe      332         56     53164                  6   
11       regedit.exe      332         28      4128                  6   
12      setup_wm.exe      332         28      4192 

" \ndf2 = pd.read_csv('data_ransomware.csv', sep=',')\nresult = pd.concat([df, df2], axis=0)\ndf = result\n"

In [7]:
import pandas as pd
from sklearn import model_selection
import sklearn.ensemble as ske
import sklearn.metrics
from sklearn.metrics import f1_score, classification_report

import disassembler # python file that fetches hex codes

#combine benign samples' hexcode and pe result
hexcodes_output_file = "hexcodes_benign.csv"
df_pe_benign = pd.read_csv(output_file_benign, sep=',')
disassembler.disassemble_files("BenignTestData", hexcodes_output_file)
df_hexcodes_benign = pd.read_csv(hexcodes_output_file, sep=',')

df_benign = pd.concat([df_pe_benign, df_hexcodes_benign], axis=1)
print(df_benign)
##############################################

#combine ransom samples' hexcode and pe reslt
hexcodes_output_file = "hexcodes_ransom.csv"
df_pe_ransom = pd.read_csv(output_file_ransomware, sep=',')
disassembler.disassemble_files("RansomwareTestData", hexcodes_output_file)
df_hexcodes_ransom = pd.read_csv(hexcodes_output_file, sep=',')

df_ransom = pd.concat([df_pe_ransom, df_hexcodes_ransom], axis=1)
print(df_ransom)
##############################################

#combined dataframe of ransom and benign samples
df = pd.concat([df_benign, df_ransom], axis=0)
print(df)


calc.exe
Camera.exe
cmd.exe
dfrgui.exe
explorer.exe
FileManager.exe
mspaint.exe
msra.exe
notepad.exe
PhotosApp.exe
reg.exe
regedit.exe
setup_wm.exe
SnippingTool.exe
StikyNot.exe
TRACERT.EXE
vds.exe
            FileName  Machine  DebugSize  DebugRVA  MajorImageVersion  \
0           calc.exe      332         56    376204                  6   
1         Camera.exe      332         56    500544                  0   
2            cmd.exe      332         56    151160                  6   
3         dfrgui.exe      332         28      4128                  6   
4       explorer.exe      332         56   1121668                  6   
5    FileManager.exe      332         56     24976                  0   
6        mspaint.exe      332         56    592320                  6   
7           msra.exe      332         28      4160                  6   
8        notepad.exe      332         56     90412                  6   
9      PhotosApp.exe      332         56     24976                  0   

FileNotFoundError: [Errno 2] No such file or directory: 'data_ransomware.csv'

In [8]:
df_temp = pd.concat([df_hexcodes_benign, df_hexcodes_benign], axis=0)
print(df_temp)

        00     01    02    03     04    05    06    07     08    09  ...  \
0    49619   5335  2557  1939   6465  1518  1447   978   4217   581  ...   
1    59907   4309  1997  1431   7351  1023  7383  2166   8793   481  ...   
2    17425   1637  1644   559   1100   709   396   319   1110   162  ...   
3    12580    659   397   250    594   261   207   311    525   143  ...   
4   144876  11301  6199  5452  10129  3764  3653  3593  13444  1399  ...   
5     1710    115    39    43    267    36    94    36    302    16  ...   
6    89371   7239  3768  3127   6857  2489  1589  1233   7950   863  ...   
7    52212   3536  2229  1168   2457  1006   816  1243   3206   440  ...   
8    12554   1066   613   574    723   463   219   315    661   218  ...   
9     1710    115    39    43    268    36    94    36    302    16  ...   
10    6842    612   386   267    377   178   100   113    375    65  ...   
11   13010   1056   617   451    804   471   244   332    656   184  ...   
12   38147  

Training and testing parts, uses Random Forest

In [12]:
print("Training the dataset using Random Forest...")

# Drops FileName and Label from data.
X = df.drop(['FileName', 'Benign'], axis=1).values


# Assigns y to label
y = df['Benign'].values

# Splitting data into training and test data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

# Print the number of training and testing samples.
print("\n\t[*] Training samples: ", len(X_train))
print("\t[*] Testing samples: ", len(X_test))

# Train Random forest algorithm on training dataset.
clf = ske.RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)
''' 
# predictions
rfc_predict = clf.predict(X_test)
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
'''

# Perform cross validation and print out accuracy.
score = model_selection.cross_val_score(clf, X_test, y_test, cv=3)
print("\n\t[*] Cross Validation Score: ", round(score.mean()*100, 2), '%')

# Calculate f1 score.
y_train_pred = model_selection.cross_val_predict(clf, X_train, y_train, cv=3)
f = f1_score(y_train, y_train_pred)
print("\t[*] F1 Score: ", round(f*100, 2), '%')


Training the dataset using Random Forest...
(13, 271)

	[*] Training samples:  13
	[*] Testing samples:  4

	[*] Cross Validation Score:  100.0 %
	[*] F1 Score:  100.0 %
